In [2]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
from __future__ import annotations
from keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM, Dropout
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from keras_preprocessing.sequence import pad_sequences

In [2]:
# 토크나이저 생성
tokenizer = Okt()

# 데이터셋 불러오기
train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test.csv")
title_name=pd.read_csv("../data/music_info.csv")

In [3]:
train_data

,Unnamed: 0,lyric,label
0,0,생각이 많은 건 말이야,0
1,1,당연히 해야 할 일이야,1
2,2,나에겐 우리가 지금 1순위야,1
3,3,안전한 유리병을 핑계로,-1
4,4,바람을 가둬 둔 것 같지만,1
...,...,...,...
2318,3187,널 위해 살아갈게,1
2319,3188,나약한 마음 따윈 모두 버릴게,1
2320,3189,우리의 사랑을 위해,1
2321,3190,너의 손을 잡고 놓지 않을게,-1


In [13]:
# 불용어 사전 로드
file_path="../words/stopwords-ko.txt"

with open(file_path,encoding='utf-8') as f:
  lines = f.readlines()
lines = [line.rstrip('\n') for line in lines]

stopwords = lines

# 훈련 데이터 전처리
okt = Okt()
X_train = []
for lyrics in train_data['lyric']:
  temp_X = []
  temp_X = okt.morphs(lyrics, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_train.append(temp_X)

# 테스트 데이터 전처리
X_test = []
for lyrics in test_data['lyrics']:
  temp_X = []
  temp_X = okt.morphs(lyrics, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

# 토큰화된 데이터의 일부 출력
print(X_train[100:103])
print(X_test[100:103])

[['손자', '돌려주다', '꼭'], ['난', '내', '놈', '씩'], ['이루다', '점점', '끝없이']]
[['짓', '없이', '그냥', '말', '만', '많다'], ['제발', '그만', '나오다', '"'], ['"', '노래', '못', '싱잉']]


In [14]:
X_train

[['생각', '많다', '건', '말', '이야'],
 ['당연하다', '일이'],
 ['에겐', '지금', '1', '순위'],
 ['안전하다', '유리', '병', '핑계'],
 ['바람', '가두다', '두다'],
 ['기억나다', '?', '그날'],
 ['잡다', '손', '엔', '말', '이야'],
 ['설레임', '보다', '커다랗다', '믿음', '담기다'],
 ['난', '함박', '웃음', '지다'],
 ['울음', '날'],
 ['소중하다', '건', '언제나', '두려움', '이니까'],
 ['문', '열다', '들리다', '목소리'],
 ['인하다', '변하다', '따뜻하다', '공기'],
 ['여전하다', '없다', '안녕하다'],
 [',', '사라지다', '별', '자리'],
 ['아스라이', '하얗다', '빛'],
 ['한', '꺼내다', '볼', '거야'],
 ['아낌없이', '반짝이다'],
 ['조금씩', '옅다', '가다'],
 ['내', '맘', '살', '숨', '쉬다', '테', '니'],
 ['여기다', ',', '서로', '끝', '아니다'],
 ['새롭다', '길', '모퉁이'],
 ['익숙하다', '진심', '속', '이지', '말'],
 ['추억', '떠오르다'],
 ['많이', '많이', '그립다', '거야'],
 ['고맙다', '이제'],
 ['사건', '지평선', '너머'],
 ['솔직하다', '두렵다', '기도'],
 ['노력', '정답', '아니다'],
 ['마지막', '선물', '산뜻하다', '안녕'],
 ['가렵지', '않다'],
 ['내', '뒤', '말', '많다'],
 ['말', '더', '필요하다'],
 ['무시', '마', '내', '걸어오다', '커리어'],
 ['더', '높이', '가줄'],
 ['내', '바르다', '세계', '젤', '위'],
 ['떨어지다', '돼다'],
 ['멋대로', '정', '나다', '애', '대해'],
 ['뜨겁다', '관심', '환영'],
 ['내',

In [4]:
# 토큰화 설정 및 토큰화 진행
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test) 
print(X_train[100:103])
print(X_test[100:103])

[[1113, 1114, 189], [5, 2, 332, 270], [415, 276, 721]]
[[1, 863, 80, 127, 10, 16, 85], [1895, 883, 129, 974], [974, 109, 9, 13, 1, 1]]


In [5]:
# 라벨 데이터 준비
y_train = []
y_test = []

# 원-핫 인코딩
for i in range(len(train_data['label'])):
  if train_data['label'].iloc[i] == 1:
    y_train.append([0, 0, 1])
  elif train_data['label'].iloc[i] == 0:
    y_train.append([0, 1, 0])
  elif train_data['label'].iloc[i] == -1:
    y_train.append([1, 0, 0])

for i in range(len(test_data['label'])):
  if test_data['label'].iloc[i] == 1:
    y_test.append([0, 0, 1])
  elif test_data['label'].iloc[i] == 0:
    y_test.append([0, 1, 0])
  elif test_data['label'].iloc[i] == -1:
    y_test.append([1, 0, 0])

y_train = np.array(y_train)
y_test = np.array(y_test)

print(y_train[100:103])
print(y_test[100:103])

[[0 0 1]
 [0 0 1]
 [1 0 0]]
[[0 1 0]
 [0 0 1]
 [0 0 1]]


## 기존 모델
- 원핫인코딩 이용
- LSTM (64개 unit, hidden-layer 1개)
- optimizer: adam
- loss: categorical_crossentropy
- 조기종료 patience=5

In [6]:
# 데이터 길이 맞추기
max_len = 12 # 전체 데이터의 길이를 12로 맞춤
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [6]:
# 모델 구성
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [7]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 9s 86ms/step - loss: 1.0443 - accuracy: 0.4693 - val_loss: 1.0128 - val_accuracy: 0.5548
Epoch 2/20
59/59 [==============================] - 5s 79ms/step - loss: 0.9773 - accuracy: 0.5560 - val_loss: 0.9176 - val_accuracy: 0.5720
Epoch 3/20
59/59 [==============================] - 5s 79ms/step - loss: 0.7381 - accuracy: 0.6733 - val_loss: 0.9150 - val_accuracy: 0.5548
Epoch 4/20
59/59 [==============================] - 5s 84ms/step - loss: 0.4618 - accuracy: 0.8428 - val_loss: 0.8415 - val_accuracy: 0.6387
Epoch 5/20
59/59 [==============================] - 5s 82ms/step - loss: 0.2985 - accuracy: 0.9090 - val_loss: 0.9649 - val_accuracy: 0.6452
Epoch 6/20
59/59 [==============================] - 5s 82ms/step - loss: 0.1889 - accuracy: 0.9349 - val_loss: 1.1812 - val_accuracy: 0.6000
Epoch 7/20
59/59 [==============================] - 5s 89ms/step - loss: 0.1526 - accuracy: 0.9456 - val_loss: 1.1203 - val_accuracy: 0.6129
Epoch 8/20
59

In [8]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 4ms/step - loss: 0.5042 - accuracy: 0.8543

 테스트 정확도 : 85.43%


# 1. optimizer 변경
- 기존 모델: Adam 사용
- 실험: Nadam, SGD, RmsProp

## 1.1) Nadam

In [7]:
# 모델 컴파일
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 6s 53ms/step - loss: 1.0506 - accuracy: 0.4666 - val_loss: 1.0147 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 3s 51ms/step - loss: 0.9987 - accuracy: 0.4849 - val_loss: 0.9772 - val_accuracy: 0.5699
Epoch 3/20
59/59 [==============================] - 3s 49ms/step - loss: 0.8328 - accuracy: 0.6399 - val_loss: 0.9750 - val_accuracy: 0.6129
Epoch 4/20
59/59 [==============================] - 3s 55ms/step - loss: 0.6005 - accuracy: 0.7691 - val_loss: 0.9118 - val_accuracy: 0.5269
Epoch 5/20
59/59 [==============================] - 3s 52ms/step - loss: 0.3942 - accuracy: 0.8687 - val_loss: 0.8533 - val_accuracy: 0.6538
Epoch 6/20
59/59 [==============================] - 3s 51ms/step - loss: 0.2760 - accuracy: 0.9112 - val_loss: 1.0612 - val_accuracy: 0.5871
Epoch 7/20
59/59 [==============================] - 3s 54ms/step - loss: 0.2380 - accuracy: 0.9273 - val_loss: 1.0516 - val_accuracy: 0.6022
Epoch 8/20
59

In [8]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 3ms/step - loss: 0.8940 - accuracy: 0.7155

 테스트 정확도 : 71.55%


## 1.2) SGD

In [11]:
# 모델 컴파일
from keras.optimizers import gradient_descent_v2

sgd = gradient_descent_v2.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 4s 21ms/step - loss: 1.0709 - accuracy: 0.4736 - val_loss: 1.0399 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 1s 14ms/step - loss: 1.0470 - accuracy: 0.4736 - val_loss: 1.0218 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 1s 15ms/step - loss: 1.0401 - accuracy: 0.4736 - val_loss: 1.0166 - val_accuracy: 0.5419
Epoch 4/20
59/59 [==============================] - 1s 16ms/step - loss: 1.0356 - accuracy: 0.4736 - val_loss: 1.0153 - val_accuracy: 0.5419
Epoch 5/20
59/59 [==============================] - 1s 16ms/step - loss: 1.0372 - accuracy: 0.4736 - val_loss: 1.0159 - val_accuracy: 0.5419
Epoch 6/20
59/59 [==============================] - 1s 14ms/step - loss: 1.0359 - accuracy: 0.4736 - val_loss: 1.0158 - val_accuracy: 0.5419
Epoch 7/20
59/59 [==============================] - 1s 15ms/step - loss: 1.0366 - accuracy: 0.4736 - val_loss: 1.0122 - val_accuracy: 0.5419
Epoch 8/20
59

In [12]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 3ms/step - loss: 1.0301 - accuracy: 0.4954

 테스트 정확도 : 49.54%


## 1.3) RmsProp

In [7]:
# 모델 컴파일
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 6s 44ms/step - loss: 1.0427 - accuracy: 0.4726 - val_loss: 0.9914 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 2s 36ms/step - loss: 0.9796 - accuracy: 0.4855 - val_loss: 0.9507 - val_accuracy: 0.5484
Epoch 3/20
59/59 [==============================] - 2s 36ms/step - loss: 0.8521 - accuracy: 0.6383 - val_loss: 1.0152 - val_accuracy: 0.5978
Epoch 4/20
59/59 [==============================] - 3s 44ms/step - loss: 0.6956 - accuracy: 0.7449 - val_loss: 1.1372 - val_accuracy: 0.6065
Epoch 5/20
59/59 [==============================] - 2s 38ms/step - loss: 0.5714 - accuracy: 0.7987 - val_loss: 1.0243 - val_accuracy: 0.4796
Epoch 6/20
59/59 [==============================] - 3s 43ms/step - loss: 0.4510 - accuracy: 0.8412 - val_loss: 0.8726 - val_accuracy: 0.6430
Epoch 7/20
59/59 [==============================] - 2s 38ms/step - loss: 0.3592 - accuracy: 0.8746 - val_loss: 0.9122 - val_accuracy: 0.6258
Epoch 8/20
59

In [8]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 5ms/step - loss: 0.6209 - accuracy: 0.8327

 테스트 정확도 : 83.27%


Adam과 Nadam, SGD, RmsProp을 비교한 결과 Adam에서의 정확도가 85.43%로 가장 높은 것을 확인할 수 있었다. 따라서, 최적의 optimizer는 Adam이라고 할 수 있을 것이다.

# 2. loss function
- 기존 모델: categorical crossentropy
- 실험: MSE, MAE, Huber loss

## 2.1) MSE

In [7]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 9s 86ms/step - loss: 0.2106 - accuracy: 0.4666 - val_loss: 0.2000 - val_accuracy: 0.5441
Epoch 2/20
59/59 [==============================] - 4s 76ms/step - loss: 0.1930 - accuracy: 0.5377 - val_loss: 0.1816 - val_accuracy: 0.6043
Epoch 3/20
59/59 [==============================] - 4s 75ms/step - loss: 0.1435 - accuracy: 0.6878 - val_loss: 0.1641 - val_accuracy: 0.6043
Epoch 4/20
59/59 [==============================] - 4s 75ms/step - loss: 0.0903 - accuracy: 0.8418 - val_loss: 0.1735 - val_accuracy: 0.6409
Epoch 5/20
59/59 [==============================] - 5s 79ms/step - loss: 0.0590 - accuracy: 0.8977 - val_loss: 0.1682 - val_accuracy: 0.6108
Epoch 6/20
59/59 [==============================] - 5s 78ms/step - loss: 0.0368 - accuracy: 0.9327 - val_loss: 0.1730 - val_accuracy: 0.6452
Epoch 7/20
59/59 [==============================] - 5s 81ms/step - loss: 0.0262 - accuracy: 0.9537 - val_loss: 0.1778 - val_accuracy: 0.6409
Epoch 8/20
59

In [8]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 3ms/step - loss: 0.0793 - accuracy: 0.8416

 테스트 정확도 : 84.16%


## 2.2) MAE

In [7]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 10s 92ms/step - loss: 0.3837 - accuracy: 0.4672 - val_loss: 0.3056 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 5s 81ms/step - loss: 0.3511 - accuracy: 0.4736 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 6s 95ms/step - loss: 0.3509 - accuracy: 0.4736 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 4/20
59/59 [==============================] - 4s 75ms/step - loss: 0.3508 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 5/20
59/59 [==============================] - 5s 79ms/step - loss: 0.3507 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 6/20
59/59 [==============================] - 5s 84ms/step - loss: 0.3507 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 7/20
59/59 [==============================] - 6s 98ms/step - loss: 0.3507 - accuracy: 0.4742 - val_loss: 0.3054 - val_accuracy: 0.5419
Epoch 8/20
5

In [8]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 5ms/step - loss: 0.3357 - accuracy: 0.4965

 테스트 정확도 : 49.65%


## 2.3) Huber loss

In [7]:
# 모델 컴파일
model.compile(optimizer='adam', loss='huber_loss', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 9s 91ms/step - loss: 0.1055 - accuracy: 0.4645 - val_loss: 0.0988 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 5s 80ms/step - loss: 0.0977 - accuracy: 0.5199 - val_loss: 0.0926 - val_accuracy: 0.6151
Epoch 3/20
59/59 [==============================] - 5s 84ms/step - loss: 0.0745 - accuracy: 0.6765 - val_loss: 0.0824 - val_accuracy: 0.6215
Epoch 4/20
59/59 [==============================] - 5s 86ms/step - loss: 0.0488 - accuracy: 0.8068 - val_loss: 0.0818 - val_accuracy: 0.6237
Epoch 5/20
59/59 [==============================] - 5s 86ms/step - loss: 0.0291 - accuracy: 0.8907 - val_loss: 0.0921 - val_accuracy: 0.5935
Epoch 6/20
59/59 [==============================] - 5s 83ms/step - loss: 0.0235 - accuracy: 0.9225 - val_loss: 0.0969 - val_accuracy: 0.6280
Epoch 7/20
59/59 [==============================] - 5s 81ms/step - loss: 0.0174 - accuracy: 0.9354 - val_loss: 0.0982 - val_accuracy: 0.6108
Epoch 8/20
59

In [8]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 6ms/step - loss: 0.0385 - accuracy: 0.8539

 테스트 정확도 : 85.39%


Categorical cross-entropy를 사용할 때의 정확도가 다른 실험에서의 정확도보다 높은 85.43%를 보이고 있다. 따라서 최적의 loss function은 categorical cross-entropy라고 볼 수 있다.

# 3. LSTM architecture 변경

## 3.1) hidden layer 수 변경

### 3.1.1) hidden layer = 2

In [8]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(64, return_sequences=False, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [9]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 12s 80ms/step - loss: 1.0439 - accuracy: 0.4672 - val_loss: 1.0142 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 4s 73ms/step - loss: 1.0275 - accuracy: 0.4736 - val_loss: 0.9971 - val_accuracy: 0.5462
Epoch 3/20
59/59 [==============================] - 4s 75ms/step - loss: 0.8939 - accuracy: 0.6200 - val_loss: 0.8834 - val_accuracy: 0.5914
Epoch 4/20
59/59 [==============================] - 4s 72ms/step - loss: 0.5686 - accuracy: 0.7777 - val_loss: 0.8460 - val_accuracy: 0.6258
Epoch 5/20
59/59 [==============================] - 5s 77ms/step - loss: 0.3446 - accuracy: 0.8708 - val_loss: 0.9479 - val_accuracy: 0.6667
Epoch 6/20
59/59 [==============================] - 5s 81ms/step - loss: 0.2493 - accuracy: 0.9047 - val_loss: 1.0102 - val_accuracy: 0.6409
Epoch 7/20
59/59 [==============================] - 4s 76ms/step - loss: 0.1902 - accuracy: 0.9333 - val_loss: 1.0896 - val_accuracy: 0.6452
Epoch 8/20
5

In [10]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 11ms/step - loss: 0.5339 - accuracy: 0.8500

 테스트 정확도 : 85.00%


### 3.1.2) hidden layer = 3

In [7]:
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.2))
model.add(LSTM(64, return_sequences=False, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 21s 129ms/step - loss: 1.0489 - accuracy: 0.4720 - val_loss: 1.0305 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 6s 109ms/step - loss: 1.0329 - accuracy: 0.4736 - val_loss: 1.0123 - val_accuracy: 0.5419
Epoch 3/20
59/59 [==============================] - 6s 110ms/step - loss: 0.9704 - accuracy: 0.5361 - val_loss: 0.9072 - val_accuracy: 0.6086
Epoch 4/20
59/59 [==============================] - 7s 113ms/step - loss: 0.7377 - accuracy: 0.6776 - val_loss: 0.8380 - val_accuracy: 0.6301
Epoch 5/20
59/59 [==============================] - 6s 109ms/step - loss: 0.5252 - accuracy: 0.7826 - val_loss: 0.8681 - val_accuracy: 0.6151
Epoch 6/20
59/59 [==============================] - 7s 122ms/step - loss: 0.3622 - accuracy: 0.8660 - val_loss: 0.9805 - val_accuracy: 0.6409
Epoch 7/20
59/59 [==============================] - 7s 115ms/step - loss: 0.2745 - accuracy: 0.8994 - val_loss: 1.1393 - val_accuracy: 0.6624
Epoch

In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 8ms/step - loss: 0.5516 - accuracy: 0.8450

 테스트 정확도 : 84.50%


## 3.2) LSTM unit 수 변경

### 3.2.1) unit=32

In [7]:
# 모델 구성
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(32,recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 8s 80ms/step - loss: 1.0568 - accuracy: 0.4629 - val_loss: 1.0152 - val_accuracy: 0.5462
Epoch 2/20
59/59 [==============================] - 4s 70ms/step - loss: 0.9855 - accuracy: 0.4984 - val_loss: 0.9373 - val_accuracy: 0.5871
Epoch 3/20
59/59 [==============================] - 5s 78ms/step - loss: 0.7803 - accuracy: 0.6674 - val_loss: 0.8116 - val_accuracy: 0.6387
Epoch 4/20
59/59 [==============================] - 5s 80ms/step - loss: 0.5022 - accuracy: 0.8283 - val_loss: 0.7903 - val_accuracy: 0.6774
Epoch 5/20
59/59 [==============================] - 5s 81ms/step - loss: 0.3395 - accuracy: 0.8961 - val_loss: 0.8722 - val_accuracy: 0.6022
Epoch 6/20
59/59 [==============================] - 5s 79ms/step - loss: 0.2423 - accuracy: 0.9338 - val_loss: 0.9952 - val_accuracy: 0.6387
Epoch 7/20
59/59 [==============================] - 5s 77ms/step - loss: 0.1673 - accuracy: 0.9532 - val_loss: 1.0652 - val_accuracy: 0.6043
Epoch 8/20
59

In [9]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 0s 5ms/step - loss: 0.5569 - accuracy: 0.8473

 테스트 정확도 : 84.73%


### 3.2.2) unit=128

In [10]:
# 모델 구성
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [11]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
59/59 [==============================] - 9s 93ms/step - loss: 1.0449 - accuracy: 0.4661 - val_loss: 1.0090 - val_accuracy: 0.5419
Epoch 2/20
59/59 [==============================] - 5s 89ms/step - loss: 0.9782 - accuracy: 0.5307 - val_loss: 0.9152 - val_accuracy: 0.5892
Epoch 3/20
59/59 [==============================] - 5s 92ms/step - loss: 0.7074 - accuracy: 0.7379 - val_loss: 0.8284 - val_accuracy: 0.6151
Epoch 4/20
59/59 [==============================] - 5s 91ms/step - loss: 0.3968 - accuracy: 0.8789 - val_loss: 0.8587 - val_accuracy: 0.6280
Epoch 5/20
59/59 [==============================] - 5s 88ms/step - loss: 0.2473 - accuracy: 0.9144 - val_loss: 0.9548 - val_accuracy: 0.6409
Epoch 6/20
59/59 [==============================] - 5s 90ms/step - loss: 0.1678 - accuracy: 0.9456 - val_loss: 1.2535 - val_accuracy: 0.6366
Epoch 7/20
59/59 [==============================] - 5s 89ms/step - loss: 0.1434 - accuracy: 0.9499 - val_loss: 1.1810 - val_accuracy: 0.6237
Epoch 8/20
59

In [12]:
# 모델 테스트 및 예측
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test)[1]*100))
predict = model.predict(X_test)

82/82 [==============================] - 1s 6ms/step - loss: 0.5445 - accuracy: 0.8462

 테스트 정확도 : 84.62%


모델의 layer 수와 unit 수를 변경해가며 실험한 결과, 초기의 layer=1, unit=64일 때 최적의 결과가 나타난다는 것을 관찰할 수 있었다.

# 4. 모델 변경

## 4.1) SVM

In [31]:
from sklearn import datasets, svm, pipeline

In [33]:
linear_svm = svm.LinearSVC()
poly_svm = svm.SVC(kernel = 'poly')
rbf_svm = svm.SVC(kernel = 'rbf')
sigmoid_svm = svm.SVC(kernel = 'sigmoid')

In [75]:
from sklearn.metrics import accuracy_score

In [76]:
# Linear Kernel
linear_svm.fit(X_train, train_data['label'])
y_pred = linear_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

# Polynomial Kernel (default:3)
poly_svm.fit(X_train, train_data['label'])
y_pred = poly_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

# RBF Kernel
rbf_svm.fit(X_train, train_data['label'])
y_pred = rbf_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

# Sigmoid Kernel
sigmoid_svm.fit(X_train, train_data['label'])
y_pred = sigmoid_svm.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

C:\Users\이하은\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.3805
Accuracy: 0.5197
Accuracy: 0.5274
Accuracy: 0.3635


SVM 모델을 이용한 경우 정확도는 LSTM을 이용했을 때보다 떨어지는 것을 확인할 수 있었다.

## 4.2) Logistic Regression

In [77]:
from sklearn.linear_model import LogisticRegression

In [81]:
lr_tf = LogisticRegression(max_iter=3000) 
lr_tf.fit(X_train, train_data['label']) # 학습
y_pred = lr_tf.predict(X_test)
 
from sklearn.metrics import accuracy_score
print('Misclassified samples: {} out of {}'.format((y_pred != test_data['label']).sum(),len(test_data['label'])))
print('Accuracy: %.4f' % accuracy_score(test_data['label'], y_pred))

Misclassified samples: 1300 out of 2594
Accuracy: 0.4988


Logistic Regression을 이용한 경우에도 역시 정확도가 0.4988로 LSTM을 이용했을 때보다 확연히 떨어지는 것을 확인할 수 있다.

# 5. Word2Vec 이용해 단어 사전 구축해보기

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [16]:
train_data['lyric']

0           생각이 많은 건 말이야
1           당연히 해야 할 일이야
2        나에겐 우리가 지금 1순위야
3           안전한 유리병을 핑계로
4         바람을 가둬 둔 것 같지만
              ...       
2318           널 위해 살아갈게
2319    나약한 마음 따윈 모두 버릴게
2320          우리의 사랑을 위해
2321     너의 손을 잡고 놓지 않을게
2322       사랑하는 내 사랑 바보야
Name: lyric, Length: 2323, dtype: object

In [92]:
pip install -U gensim --user

  Using cached gensim-4.3.2-cp38-cp38-win_amd64.whl.metadata (8.5 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl (42.2 MB)
Using cached gensim-4.3.2-cp38-cp38-win_amd64.whl (24.0 MB)
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [29]:
from gensim.models import Word2Vec

w2v = Word2Vec(sentences = X_train, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [30]:
w2v.wv.vectors.shape

(381, 100)

In [19]:
w2v.wv.key_to_index

{'내': 0,
 '보다': 1,
 '난': 2,
 '사랑': 3,
 '없다': 4,
 '말': 5,
 '날': 6,
 '못': 7,
 '더': 8,
 '만': 9,
 '널': 10,
 '되다': 11,
 '처럼': 12,
 '않다': 13,
 '안': 14,
 '싶다': 15,
 '눈': 16,
 '사람': 17,
 '좋다': 18,
 '밤': 19,
 '이제': 20,
 '가다': 21,
 '다시': 22,
 '그대': 23,
 '한': 24,
 '아니다': 25,
 ',': 26,
 '니': 27,
 '마음': 28,
 '걸': 29,
 '맘': 30,
 '나르다': 31,
 '모르다': 32,
 '위': 33,
 '해': 34,
 '모든': 35,
 '들다': 36,
 '순간': 37,
 '지금': 38,
 '버리다': 39,
 '돼다': 40,
 '거야': 41,
 '오다': 42,
 '내다': 43,
 '알다': 44,
 '거': 45,
 '손': 46,
 '그렇다': 47,
 '나다': 48,
 '꿈': 49,
 '마': 50,
 '자다': 51,
 '속': 52,
 '세상': 53,
 '생각': 54,
 '너무': 55,
 '만나다': 56,
 '보이다': 57,
 '잊다': 58,
 '끝': 59,
 '웃다': 60,
 '보고': 61,
 '아프다': 62,
 '이야': 63,
 '주다': 64,
 '어떻다': 65,
 '우린': 66,
 '많이': 67,
 '고': 68,
 '건': 69,
 '믿다': 70,
 '없이': 71,
 '두': 72,
 '아무': 73,
 '뭐': 74,
 '많다': 75,
 '눈물': 76,
 '행복하다': 77,
 '지': 78,
 '갈다': 79,
 '보내다': 80,
 '엔': 81,
 '아직': 82,
 '곁': 83,
 '차다': 84,
 '번': 85,
 '걸다': 86,
 '기억': 87,
 '온': 88,
 '가슴': 89,
 '젠': 90,
 '넌': 91,
 '인': 92,
 '요': 93,
 

In [17]:
import gensim
print(gensim.__version__)

4.3.2


In [20]:
print(w2v.wv.most_similar("꿈"))

[('내', 0.4150461256504059), ('되다', 0.4066603481769562), ('보고', 0.40526798367500305), ('말', 0.3982767164707184), ('날', 0.3915805220603943), ('온', 0.3908381164073944), ('인', 0.37818416953086853), ('안녕', 0.3596257269382477), ('주다', 0.3544522523880005), ('사랑', 0.35238614678382874)]


In [21]:
print(w2v.wv.most_similar("사랑"))

[('말', 0.521798312664032), ('줄', 0.4737316071987152), ('두', 0.4729493260383606), ('날', 0.46012622117996216), ('되다', 0.45006218552589417), ('내', 0.43687739968299866), ('되어다', 0.42670780420303345), ('난', 0.42235010862350464), ('없이', 0.42145469784736633), ('울', 0.41949015855789185)]


In [22]:
embedding_matrix = w2v.wv.syn0

AttributeError: 'KeyedVectors' object has no attribute 'syn0'

In [31]:
w2vdict = dict(zip(w2v.wv.index_to_key, w2v.wv.vectors)) 

In [32]:
w2vdict

{'내': array([-3.61794629e-03,  3.94915137e-03,  3.39029799e-03,  1.22293234e-02,
        -6.30810205e-03, -2.05393303e-02,  1.31342355e-02,  2.48731207e-02,
        -1.17249573e-02, -7.21749989e-03,  5.42818196e-03, -9.26140603e-03,
        -6.61095092e-03,  7.56890932e-03, -5.54946810e-03, -9.28985234e-03,
         4.66894312e-03, -7.64656253e-03, -1.13755902e-02, -2.51984149e-02,
         8.53883289e-03,  7.08351517e-03,  8.43318552e-03, -2.87784333e-03,
         1.50572485e-03, -3.08194244e-03, -4.52217087e-03, -2.97174929e-03,
        -1.65658779e-02, -3.73504986e-03, -1.70914072e-03, -1.31152815e-03,
         1.21502196e-02, -1.24133434e-02, -7.49552716e-03,  7.02533033e-03,
         7.19544245e-03, -1.41785899e-02, -4.56791418e-03, -1.80983786e-02,
        -5.47927432e-03, -4.03989572e-03, -1.23305302e-02, -3.90834082e-03,
         9.75000579e-03, -2.99715414e-03, -9.78682376e-03,  5.87005541e-03,
         1.04148854e-02,  1.36671383e-02, -3.31320427e-03, -6.24107022e-04,
       

In [34]:
embedding_matrix = w2v.wv.vectors

In [36]:
embedding_matrix.shape

(381, 100)

In [37]:
# Vectorize X_train and X_test to 2D tensor
top_words = embedding_matrix.shape[0]

# Define max lenght of sentence and number of classes (negative, neutral and positive)
mxlen = 30
nb_classes = 3

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

NameError: name 'y_train' is not defined

In [38]:
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
sequences_val = tokenizer.texts_to_sequences(X_val)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(word_index)

NameError: name 'X_val' is not defined